In [ ]:
import os
import re
import json
import joblib
import numpy as np
import pandas as pd
import textwrap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Concatenate, Bidirectional
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from nltk.tokenize import word_tokenize

# ================================================================
# Part 1: Load Resources, Preprocess Data, Build & Run the LSTM Model
# ================================================================

# Set paths for saved resources (adjust to your system)
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM_outputs'
vocab_file = os.path.join(output_dir, 'vocab_index.json')
max_lengths_file = os.path.join(output_dir, 'max_lengths.csv')
scaler_file = os.path.join(output_dir, 'scaler.pkl')
model_weights_file = os.path.join(output_dir, 'model_fold_4.h5')

# Load vocabulary index from JSON file
with open(vocab_file, 'r', encoding='utf-8') as f:
    word_index = json.load(f)

# Load maximum sequence lengths from CSV (each text input has its own maximum length)
max_lengths_df = pd.read_csv(max_lengths_file)
max_lengths = max_lengths_df['Max_Length'].tolist()

# Load standardization (scaler) tool for numeric features
scaler = joblib.load(scaler_file)

# Define vocabulary size and embedding dimension (should match training settings)
vocab_size = 2025
embedding_dim = 64

# Define new sample text and numeric data (replace with your actual sample data)
new_sample_text = [
    'water', 
    'To evaluate the frost resistance of , three prismatic specimens (100 mm × 100 mm × 400 mm) from all the concrete mixes were subjected to repeated freeze–thaw cycles in the DDR-9 computercontrolled concrete rapid freeze–thaw cycle testing machine. In addi tion, a temperature measurement specimen with a temperature sensor embedded in the central position was prepared.',
    'This can be interpreted as two possible mechanisms: (1) fly ash and Slag fill the micro-cracks of RCA and the pores of recycled aggregate concrete, making the internal structure of recycled aggregate concrete denser. (2) The product (C–S–H gel) of the pozzolanic reaction furtherly fills the pores of recycled aggregate concrete, also improving the inner structure of recycled aggregate concrete. So does the filling effect of the fine particles on the capillary pores in recycled aggregate concrete; the further damage caused by the water entering into capillary pores was reduced during freeze–thaw cycles.',
    'which also indicated that the frost resistance of the specimens was improved. This also might be interpreted as the activity of both fly ash and Slag are effectively excited by the availability of cement hydration product Ca (OH)2, and the mutual catalysis of fly ash and Slag is beneficial to accel- erate the production of C-S-H gel, which makes the inner structure of recycled aggregate concrete denser thus improving the frost resistance of recycled aggregate concrete.'
]
new_sample_numeric = [0.44, 0, 332.35, 0, 58.65, 0, 172, 588, 624.5, 624.5, 16, 4.95, 0, 0, 0, 0, 125, 1]

# Text preprocessing function: lowercase and remove commas, periods, and colons
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# Preprocess and tokenize the new sample texts, then create padded sequences
tokenized_texts = []
padded_sequences = []
for i, text in enumerate(new_sample_text):
    processed_text = preprocess_text(text)
    tokens = word_tokenize(processed_text)
    tokenized_texts.append(tokens)
    sequence = [word_index.get(word, 0) for word in tokens]
    padded_seq = pad_sequences([sequence], maxlen=max_lengths[i], padding='post', truncating='post')[0]
    padded_sequences.append(padded_seq)

# Scale numeric data
new_sample_numeric_scaled = scaler.transform(np.array([new_sample_numeric], dtype=np.float32))

# Define the bidirectional LSTM model architecture
def create_model(max_lengths, vocab_size, embedding_dim):
    inputs = []
    final_states = []
    sequence_outputs = []

    # Process each text input separately
    for i, max_len in enumerate(max_lengths):
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                                    input_length=max_len, trainable=True, mask_zero=True,
                                    name=f"embedding_{i+1}")(input_layer)
        # Use a Bidirectional LSTM with return_sequences and return_state=True
        lstm_layer = Bidirectional(LSTM(16, return_sequences=True, return_state=True), 
                                   name=f"bilstm_{i+1}")
        # For Bidirectional LSTM, the outputs are: [sequence_output, forward_h, forward_c, backward_h, backward_c]
        lstm_outputs = lstm_layer(embedding_layer)
        sequence_output, forward_h, _, backward_h, _ = lstm_outputs
        # Concatenate the forward and backward hidden states to form the final state
        final_state = Concatenate(name=f"final_state_{i+1}")([forward_h, backward_h])
        inputs.append(input_layer)
        final_states.append(final_state)
        sequence_outputs.append(sequence_output)
    
    # Process numerical features
    input_numerical = Input(shape=(18,), dtype='float32', name="numerical_input")
    inputs.append(input_numerical)
    
    # Combine final states from texts and numeric input
    combined_features = Concatenate(name="combined_features")(final_states + [input_numerical])
    x = Dense(128, activation='relu', kernel_initializer=HeNormal(), name="dense_1")(combined_features)
    x = Dropout(0.2, name="dropout_1")(x)
    x = Dense(64, activation='relu', kernel_initializer=HeNormal(), name="dense_2")(x)
    x = Dropout(0.2, name="dropout_2")(x)
    x = Dense(32, activation='relu', kernel_initializer=HeNormal(), name="dense_3")(x)
    x = Dropout(0.2, name="dropout_3")(x)
    final_outputs = Dense(2, name="final_output")(x)
    
    # Create model with primary output for prediction and sequence outputs for analysis
    model = Model(inputs=inputs, outputs=[final_outputs] + sequence_outputs, name="analysis_model")
    # Compile model; only compute loss on the primary output
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss={'final_output': 'mae'}, loss_weights={'final_output': 1.0})
    return model

# Build the model and load the saved weights
model = create_model(max_lengths, vocab_size, embedding_dim)
model.load_weights(model_weights_file)

# Prepare inputs (reshape text sequences and append numeric input)
text_inputs = [seq.reshape(1, -1) for seq in padded_sequences]
all_inputs = text_inputs + [new_sample_numeric_scaled]

# Run prediction
model_outputs = model.predict(all_inputs)

# Extract primary prediction and sequence (hidden state) outputs
final_prediction = model_outputs[0]
sequence_outputs = model_outputs[1:]  # list of sequence outputs for each text input
print("Final Prediction Output:")
print(final_prediction)
for i, seq_output in enumerate(sequence_outputs):
    print(f"Sequence output {i + 1} shape:", seq_output.shape)

# ================================================================
# Part 2: Hidden State Distance Computation and Export to Excel
# ================================================================
# For each text input's sequence output, compute Manhattan and cosine distances between consecutive tokens.
# The distances are computed separately for forward and backward hidden states.
# Then, the distances are standardized and normalized before being saved to an Excel file
# (with separate sheets for forward and backward distances).

for i, seq_output in enumerate(sequence_outputs):
    # Remove the batch dimension (assumes batch size = 1)
    seq_output = seq_output[0]
    tokens = tokenized_texts[i]
    actual_length = len(tokens)
    # Remove padded tokens
    seq_output = seq_output[:actual_length]

    # Split hidden states into forward and backward parts (assuming 16 units per direction)
    forward_hidden = seq_output[:, :16]
    backward_hidden = seq_output[:, 16:]

    # Initialize distance arrays
    forward_manhattan = [0] * len(tokens)
    forward_cosine = [0] * len(tokens)
    backward_manhattan = [0] * len(tokens)
    backward_cosine = [0] * len(tokens)

    # Compute distances for forward hidden states
    for j in range(1, len(forward_hidden)):
        manhattan_dist = np.sum(np.abs(forward_hidden[j] - forward_hidden[j-1]))
        forward_manhattan[j] = manhattan_dist
        dot_prod = np.dot(forward_hidden[j-1], forward_hidden[j])
        norm_prod = np.linalg.norm(forward_hidden[j-1]) * np.linalg.norm(forward_hidden[j])
        cosine_dist = 1 - (dot_prod / norm_prod) if norm_prod != 0 else 0
        forward_cosine[j] = cosine_dist

    # Compute distances for backward hidden states
    for j in range(1, len(backward_hidden)):
        manhattan_dist = np.sum(np.abs(backward_hidden[j] - backward_hidden[j-1]))
        backward_manhattan[j] = manhattan_dist
        dot_prod = np.dot(backward_hidden[j], backward_hidden[j-1])
        norm_prod = np.linalg.norm(backward_hidden[j]) * np.linalg.norm(backward_hidden[j-1])
        cosine_dist = 1 - (dot_prod / norm_prod) if norm_prod != 0 else 0
        backward_cosine[j] = cosine_dist

    # Reverse the backward distances for alignment with the forward order
    backward_manhattan_reversed = forward_manhattan[::-1]
    backward_cosine_reversed = backward_cosine[::-1]

    # Create DataFrames to store forward and backward distance measurements
    df_forward = pd.DataFrame({
        'Token (Forward)': tokens,
        'Manhattan Distance (Forward)': forward_manhattan,
        'Cosine Distance (Forward)': forward_cosine
    })
    df_backward = pd.DataFrame({
        'Token (Backward)': tokens,
        'Manhattan Distance (Backward)': backward_manhattan_reversed,
        'Cosine Distance (Backward)': backward_cosine_reversed
    })

    # Standardize and normalize the computed distances (forward)
    if len(df_forward) > 1:
        standard_scaler = StandardScaler()
        forward_cols = ['Manhattan Distance (Forward)', 'Cosine Distance (Forward)']
        standardized_forward = standard_scaler.fit_transform(df_forward[forward_cols].iloc[1:])
        df_forward.loc[1:, [f'{col} (Standardized)' for col in forward_cols]] = standardized_forward

        minmax_scaler = MinMaxScaler()
        normalized_forward = minmax_scaler.fit_transform(df_forward[[f'{col} (Standardized)' for col in forward_cols]].iloc[1:])
        df_forward.loc[1:, [f'{col} (Normalized)' for col in forward_cols]] = normalized_forward

    # Standardize and normalize the computed distances (backward)
    if len(df_backward) > 1:
        standard_scaler = StandardScaler()
        backward_cols = ['Manhattan Distance (Backward)', 'Cosine Distance (Backward)']
        standardized_backward = standard_scaler.fit_transform(df_backward[backward_cols].iloc[:-1])
        df_backward.loc[:len(df_backward)-2, [f'{col} (Standardized)' for col in backward_cols]] = standardized_backward

        minmax_scaler = MinMaxScaler()
        normalized_backward = minmax_scaler.fit_transform(df_backward[[f'{col} (Standardized)' for col in backward_cols]].iloc[:-1])
        df_backward.loc[:len(df_backward)-2, [f'{col} (Normalized)' for col in backward_cols]] = normalized_backward

    # Calculate normalized distance sums if normalized columns exist
    if 'Manhattan Distance (Forward) (Normalized)' in df_forward.columns and 'Cosine Distance (Forward) (Normalized)' in df_forward.columns:
        df_forward['Distance Sum (Forward) (Normalized)'] = (df_forward['Manhattan Distance (Forward) (Normalized)'] +
                                                              df_forward['Cosine Distance (Forward) (Normalized)'])
    if 'Manhattan Distance (Backward) (Normalized)' in df_backward.columns and 'Cosine Distance (Backward) (Normalized)' in df_backward.columns:
        df_backward['Distance Sum (Backward) (Normalized)'] = (df_backward['Manhattan Distance (Backward) (Normalized)'] +
                                                               df_backward['Cosine Distance (Backward) (Normalized)'])

    # Save the DataFrames to an Excel file with separate sheets for forward and backward data
    output_file = os.path.join(output_dir, f'hidden_states_text_input_{i+1}.xlsx')
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df_forward.to_excel(writer, sheet_name='Forward', index=False)
        df_backward.to_excel(writer, sheet_name='Backward', index=False)

# ================================================================
# Part 3: Visualize Token Mapping from Excel Files
# ================================================================
# This section reads the Excel files generated above and visualizes tokens using a colormap
# based on the normalized distance scores.

def visualized_text(df, score_column, tokens, normalized_scores, cmap, wrapper, excel_file, sheet_name):
    # Create a figure with defined dimensions
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Layout parameters for text drawing
    left_margin = 0.1
    right_margin = 0.1
    canvas_width = 10  # total width (in arbitrary units)
    x_pos = left_margin
    y_start = 0.9
    y_pos = y_start
    line_height = 0.01  # reduced line spacing
    char_width = 0.15   # estimated width per character
    word_spacing = 0.3  # spacing between words
    token_idx = 0
    
    # Draw each token individually, wrapping when needed
    for token in tokens:
        word_width = len(token) * char_width
        if x_pos + word_width > (canvas_width - right_margin):
            x_pos = left_margin
            y_pos -= line_height
        # Get the normalized score for this token (or default to 0)
        if token_idx < len(normalized_scores):
            score = normalized_scores.iloc[token_idx]
        else:
            score = 0
        ax.text(x_pos, y_pos, token,
                fontsize=18,
                fontweight='bold',
                family='Times New Roman',
                color=cmap(score),
                ha='left',
                va='top')
        x_pos += word_width + word_spacing
        token_idx += 1

    ax.set_xlim(0, canvas_width)
    ax.set_ylim(y_pos - line_height, y_start + line_height)
    ax.axis('off')

    # Save the visualization as an SVG image
    save_dir = os.path.join(output_dir, "visualization")
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    base_name = os.path.basename(excel_file).replace('.xlsx', '')
    save_path = os.path.join(save_dir, f"visualized_text_output_{base_name}_{score_column.replace(' ', '_')}_{sheet_name}.svg")
    print(f"Saving visualization to: {save_path}")
    plt.savefig(save_path, format='svg', dpi=300, bbox_inches='tight')
    plt.show()

def visualized_text_from_excel(excel_files, cmap, wrapper):
    for excel_file in excel_files:
        # Read both sheets from the Excel file
        df_forward = pd.read_excel(excel_file, sheet_name='Forward')
        df_backward = pd.read_excel(excel_file, sheet_name='Backward')
        print(f"Processing file: {excel_file}")
        
        # Get token lists from the first column of each sheet
        tokens_forward = df_forward.iloc[:, 0]
        tokens_backward = df_backward.iloc[:, 0]
        
        # Get the last column (assumed normalized score) and replace NaN with 0
        forward_scores = df_forward.iloc[:, -1].fillna(0)
        backward_scores = df_backward.iloc[:, -1].fillna(0)
        
        print("Visualizing forward tokens and scores.")
        visualized_text(df_forward, forward_scores.name, tokens_forward, forward_scores, cmap, wrapper, excel_file, 'Forward')
        print("Visualizing backward tokens and scores.")
        visualized_text(df_backward, backward_scores.name, tokens_backward, backward_scores, cmap, wrapper, excel_file, 'Backward')

# List the Excel files that were created (adjust file names/paths if necessary)
excel_files = [
    os.path.join(output_dir, "hidden_states_text_input_1.xlsx"),
    os.path.join(output_dir, "hidden_states_text_input_2.xlsx"),
    os.path.join(output_dir, "hidden_states_text_input_3.xlsx"),
    os.path.join(output_dir, "hidden_states_text_input_4.xlsx")
]

# Define the colormap and text wrapper settings
cmap = plt.get_cmap('Blues')
wrapper = textwrap.TextWrapper(width=90)

# Run visualization on the Excel files
visualized_text_from_excel(excel_files, cmap, wrapper)
